In [57]:
import pandas as pd
import numpy as np
import CHARMio
import imp
imp.reload(CHARMio)

<module 'CHARMio' from '/shareb/zliu/analysis/CHARMtools/CHARMio.py'>

In [36]:
tdgpath = "/shareb/zliu/analysis/CHARM/CHARM_mesc/Rawdataprocess/hic/processed/R5Ac011/3d_info/50k.1.3dg"
tdg = CHARMio.parse_3dg(tdgpath,sorting=False,s2m=True)

In [69]:
def calculate_resolution(df: pd.DataFrame) -> int:
    """
    Calculate the resolution of a DataFrame with a MultiIndex (chr, pos).
    Resolution is defined as the smallest non-zero difference between the 'pos' values for each 'chr'.
    Args:
        df: A pandas DataFrame with a MultiIndex where the first level is 'chr' and the second level is 'pos'.
    Returns:
        The resolution as an integer.
    """
    # Initialize a large number to find the minimum resolution
    min_resolution = float('inf')

    # Group by the chromosome level of the index
    for chromosome, group in df.groupby(level=0):
        # Calculate differences between adjacent positions
        pos_diffs = group.index.get_level_values(1).to_series().diff().dropna()

        # Exclude zero differences and find the minimum
        non_zero_diffs = pos_diffs[pos_diffs != 0]
        if not non_zero_diffs.empty:
            chrom_min_res = non_zero_diffs.min()
            # Update overall minimum resolution if this chromosome's min is smaller
            min_resolution = min(min_resolution, chrom_min_res)

    # If min_resolution is still infinity, there were no non-zero resolutions found
    if min_resolution == float('inf'):
        raise ValueError("No non-zero position differences found in the DataFrame.")

    return int(min_resolution)

# 假设 df 是您的 DataFrame
# 调用此功能并传入 df
# resolution = calculate_resolution(df)
# print(resolution)

In [71]:
calculate_resolution(tdg)

50000

In [62]:
# print pandas version 
vec = tdg.index.get_level_values(1).values
# get min value of one by one value
min_adjacent_values = np.minimum(vec[:-1]-vec[1:])
#print(min_adjacent_values)

TypeError: minimum() takes from 2 to 3 positional arguments but 1 were given

In [63]:
vec[:-1]

array([    25000,     75000,    125000, ..., 170875000, 170925000,
       170975000])

In [67]:
vec[1:][:len(vec[:-1])] - vec[:-1]

array([-50000, -50000, -50000, ..., -50000, -50000, -50000])

In [41]:
def check_index_binsize(s: pd.DataFrame) -> int:
    """
    Check if the index of s, considering its first level grouping, has sorted and consistent binsizes.
    
    The function assumes that the second level of the MultiIndex is numerical (e.g., genomic coordinates or timestamps).
    It calculates the binsize for each group defined by the first level and returns a series with the binsizes.
    If any inconsistency in binsize within a group is found, raises ValueError.
    Input:
        s: A pandas DataFrame with a MultiIndex where the first level represents chromosome names and the second level
              represents positions or other values that should have a consistent difference (binsize).
    Output:
        binsize
    """

    # Ensure the index is sorted
    s = s.sort_index()
    # Calculate binsizes per group based on the first level
    # last 2 element is not used, because the last one is NaN and in some situations the second last one is partial binsize
    # negative period is used to ensure first element is not NaN
    result_dfs = []
    for name, group in s.groupby(level=0):
        new_df = pd.Series(
            -group.index.get_level_values(1).diff(-1),
            index = group.index
            ).rename("binsizes").iloc[:-2]
        result_dfs.append(new_df)
    binsizes = pd.concat(result_dfs, axis=0).dropna().astype(int)
    print(binsizes)
    if binsizes.empty:
        print("Warning: No binsize found.")
        # just use the first binsize
        binsize = -s.index.get_level_values(1).diff()[0]
        #print(binsize)
    elif len(binsizes.unique()) > 1:
        print("Warning: Inconsistent binsizes found in the input file %s" % binsizes.unique())
        binsize = binsizes.dropna().unique()[0]
        print(binsizes)
    else:
        binsize = binsizes.dropna().unique()[0]
    return binsize

In [39]:
check_index_binsize(tdg)

50000